# 1. Overview dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r"C:\Users\serale\OneDrive - OMRON\Documents\AlexSerra\TFM_AlexSerra\TFM_MBD_2024_AlexSerra\data\Times_Series_Project_uoc.xlsx")

In [3]:
df.head()

,date,sales_region1,sales_region2,sales_region3,humidity_avg_Basel,humidity_avg_Bellinzona,humidity_avg_Bern,humidity_avg_Chur,humidity_avg_Distretto di Lugano,humidity_avg_Geneva,...,gtrends_CH-VS_pollen allergie,gtrends_CH-ZH_heuschnupfen,gtrends_CH-ZH_pollen,gtrends_CH-ZH_pollen allergie,influenza,pollution_Zurich_no2,pollution_Zurich_o3,pollution_Zurich_pm10,pollution_Zurich_pm25,pollution_Zurich_so2
0,2020-01-06,0.680314,1.144905,1.680806,85.688095,80.119048,92.354497,65.678571,76.190476,83.732143,...,0,0,0,0,221.0,13.942857,5.657143,15.428571,3.000000,0.500000
1,2020-01-13,1.469124,1.033474,1.395917,83.095714,80.625000,84.059524,58.386905,77.696429,80.839286,...,0,0,0,0,3.0,16.328571,5.485714,12.285714,3.714286,0.885714
2,2020-01-20,0.948362,1.026370,2.199281,78.803810,68.232143,81.423077,68.261905,67.952381,74.791667,...,0,0,0,0,530.5,14.828571,11.000000,10.714286,2.571429,0.914286
3,2020-01-27,0.885640,1.034298,1.648535,84.359601,76.678830,89.023810,61.957557,73.318723,87.666149,...,0,0,0,0,911.5,18.314286,2.685714,24.142857,5.000000,0.785714
4,2020-02-03,1.686237,0.525493,1.296067,79.455952,81.726449,78.101967,77.494565,79.429654,83.072981,...,0,0,0,0,772.0,6.985714,20.357143,6.428571,1.285714,0.371429


In [4]:
df.shape

(183, 66)

In [5]:
df.columns

Index(['date', 'sales_region1', 'sales_region2', 'sales_region3',
       'humidity_avg_Basel', 'humidity_avg_Bellinzona', 'humidity_avg_Bern',
       'humidity_avg_Chur', 'humidity_avg_Distretto di Lugano',
       'humidity_avg_Geneva', 'humidity_avg_Lausanne', 'humidity_avg_Sion',
       'humidity_avg_Zurich', 'temp_avg_k_Basel', 'temp_avg_k_Bellinzona',
       'temp_avg_k_Bern', 'temp_avg_k_Chur', 'temp_avg_k_Distretto di Lugano',
       'temp_avg_k_Geneva', 'temp_avg_k_Lausanne', 'temp_avg_k_Sion',
       'temp_avg_k_Zurich', 'gtrends_CH_allergie',
       'gtrends_CH_pollen allergie', 'gtrends_CH-BE_allergie',
       'gtrends_CH-BE_heuschnupfen', 'gtrends_CH-BE_pollen',
       'gtrends_CH-BE_pollen allergie', 'gtrends_CH-BS_heuschnupfen',
       'gtrends_CH-BS_pollen', 'gtrends_CH-FR_allergie',
       'gtrends_CH-FR_heuschnupfen', 'gtrends_CH-GR_allergie',
       'gtrends_CH-GR_heuschnupfen', 'gtrends_CH-GR_pollen',
       'gtrends_CH-LU_allergie', 'gtrends_CH-LU_heuschnupfen',
    

# 2. Creating function that preprocess dataframe

- Keep date
- Sum Sales
- Average humidity
- Average temp
- Sum all values of same gtrends (but different locations)
- Keep same values of every polution and influenza

In [6]:
def pre_general_agg(df):
    
    df_clean=pd.DataFrame()
    
    df_clean["date"]=df["date"]
    
    l_sales = [col for col in df.columns if "sales" in col]
    df_clean["sales"]=df[l_sales].sum(axis=1)
    
    l_humidity = [col for col in df.columns if "humidity" in col]
    df_clean["humidity"]=df[l_humidity].mean(axis=1)
    
    l_temp = [col for col in df.columns if "temp" in col]
    df_clean["temp"]=df[l_temp].mean(axis=1)
    
    return(df_clean)

    

In [7]:
def pre_gtrends(df, df_clean):
    
    gtrends_columns = [col for col in df.columns if 'gtrends' in col]
    gtrend_unique_col=[]
    for e in gtrends_columns:
        val=e.split("_")[2]
        if val=="allergie":
            val="_"+val
        if val not in gtrend_unique_col:
            l=[col for col in df.columns if val in col]
            df_clean["gtrends_"+val]=df[l].sum(axis=1)
    
    return (df_clean)

In [8]:
def pre_others(df, df_clean):
    
    exclude=["date", "sales", "humidity", "temp", "gtrends"]
    unique_col=[col for col in df.columns if not any(keyword in col.lower() for keyword in exclude)]
    for e in unique_col:
        df_clean[e]=df[e]
    
    df_clean.rename(columns={'gtrends__allergie': "gtrends_allergie"}, inplace=True)
    
    return(df_clean)
    

In [9]:
def preprocessing(df):
    
    df_1 = pre_general_agg(df)
    
    df_2 = pre_gtrends(df, df_1)
    
    df_3 = pre_others(df, df_2)
    
    return df_3    

In [10]:
df_clean= preprocessing(df)

In [11]:
df_clean.head(5)

,date,sales,humidity,temp,gtrends_allergie,gtrends_pollen allergie,gtrends_heuschnupfen,gtrends_pollen,influenza,pollution_Zurich_no2,pollution_Zurich_o3,pollution_Zurich_pm10,pollution_Zurich_pm25,pollution_Zurich_so2
0,2020-01-06,3.506025,81.875606,274.946388,115,0,8,22,221.0,13.942857,5.657143,15.428571,3.000000,0.500000
1,2020-01-13,3.898515,77.805608,275.982323,176,0,36,39,3.0,16.328571,5.485714,12.285714,3.714286,0.885714
2,2020-01-20,4.174013,73.820989,275.575986,140,0,5,27,530.5,14.828571,11.000000,10.714286,2.571429,0.914286
3,2020-01-27,3.568473,80.112456,274.589907,150,0,5,83,911.5,18.314286,2.685714,24.142857,5.000000,0.785714
4,2020-02-03,3.507797,79.792778,279.608504,98,0,32,10,772.0,6.985714,20.357143,6.428571,1.285714,0.371429
